# Imports

In [1]:
import pandas as pd

# Constantes

In [2]:
path = '../datasets/raw/zomato.csv'

# Funções

In [3]:
def country_name(country_id):
    COUNTRIES = {
        1: "India",
        14: "Australia",
        30: "Brazil",
        37: "Canada",
        94: "Indonesia",
        148: "New Zeland",
        162: "Philippines",
        166: "Qatar",
        184: "Singapure",
        189: "South Africa",
        191: "Sri Lanka",
        208: "Turkey",
        214: "United Arab Emirates",
        215: "England",
        216: "United States of America"
}
    
    return COUNTRIES[country_id]

def create_price_type(price_range):
    return ['_', 'cheap', 'normal', 'expensive', 'gourmet'][price_range]

def color_name(color_code):
    COLORS = {
        "3F7E00": "darkgreen",
        "5BA829": "green",
        "9ACD32": "lightgreen",
        "CDD614": "orange",
        "FFBA00": "red",
        "CBCBC8": "darkred",
        "FF7800": "darkred"
}
    
    return COLORS[color_code]

def rename_columns(df):
    snakecase = lambda x: x.lower()
    spaces = lambda x: x.replace(" ", "_")
    cols_old = list(df.columns)
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    
    return df

def clean_data(df):
    df_clean = df.copy()
    
    df_clean = df_clean.dropna()

    df_clean = rename_columns(df_clean)
    
    df_clean['price_type'] = df_clean['price_range'].apply(lambda x: create_price_type(x))
    df_clean['rating_color_name'] = df_clean['rating_color'].apply(lambda x: color_name(x))
    df_clean['country'] = df_clean['country_code'].apply(lambda x: country_name(x))
    df_clean['cuisines'] = df_clean['cuisines'].apply(lambda x: x.split(',')[0])

    df_clean = df_clean.drop_duplicates()

    df_clean = df_clean.loc[:, [
        'restaurant_id',
        'restaurant_name',
        'country',
        'city',
        'address',
        'locality',
        'locality_verbose',
        'longitude',
        'latitude',
        'cuisines',
        'average_cost_for_two',
        'currency',
        'has_table_booking',
        'has_online_delivery',
        'is_delivering_now',
        'switch_to_order_menu',
        'price_type',
        'aggregate_rating',
        'rating_color_name',
        'rating_text',
        'votes',
    ]]

    df_clean.to_csv('../datasets/clean/zomato.csv', index=False)
    
    return df_clean

# Carga dos dados

In [4]:
df = pd.read_csv(path)

# Limpeza dos dados

In [5]:
df1 = clean_data(df)

# Perguntas de negócio

## Geral

### 1. Quantos restaurantes únicos estão registrados?

In [6]:
df1['restaurant_id'].nunique()

6929

### 2. Quantos países únicos estão registrados?

In [7]:
df1['country'].nunique()

15

### 3. Quantas cidades únicas estão registradas?

In [8]:
df1['city'].nunique()

125

### 4. Qual o total de avaliações feitas?

In [9]:
df1['votes'].sum()           

4194533

### 5. Qual o total de tipos de culinária registrados?

In [10]:
df1['cuisines'].nunique()

165

## País

### 1. Qual o nome do país que possui mais cidades registradas?

In [11]:
df1.loc[:, ['city', 'country']].groupby('country').nunique().reset_index().sort_values('city', ascending=False).iloc[0, 0]

'India'

### 2. Qual o nome do país que possui mais restaurantes registrados?

In [12]:
df1.loc[:, ['restaurant_id', 'country']].groupby('country').nunique().reset_index().sort_values('restaurant_id', ascending=False).iloc[0, 0]

'India'

### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [13]:
df1.loc[df1['price_type'] == 'gourmet', ['price_type', 'country']].groupby('country').count().reset_index().sort_values('price_type', ascending=False).iloc[0,0]

'United States of America'

### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [14]:
df1.loc[:, ['cuisines', 'country']].groupby('country').nunique().reset_index().sort_values('cuisines', ascending=False).iloc[0, 0]

'India'

### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [15]:
df1.loc[:, ['votes', 'country']].groupby('country').nunique().reset_index().sort_values('votes', ascending=False).iloc[0, 0]

'India'

### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?


In [16]:
df1.loc[df1['is_delivering_now'] == 1, ['restaurant_id', 'country']].groupby('country').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0,0]

'India'

### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [17]:
df1.loc[df1['has_table_booking'] == 1, ['restaurant_id', 'country']].groupby('country').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0,0]

'India'

### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?


In [18]:
df1.loc[:, ['votes', 'country']].groupby('country').mean().reset_index().sort_values('votes', ascending=False).iloc[0, 0]

'Indonesia'

### 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [19]:
df1.loc[:, ['aggregate_rating', 'country']].groupby('country').mean().reset_index().sort_values('aggregate_rating', ascending=False).iloc[0, 0]

'Indonesia'

### 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [20]:
df1.loc[:, ['aggregate_rating', 'country']].groupby('country').mean().reset_index().sort_values('aggregate_rating', ascending=True).iloc[0, 0]

'Brazil'

### 11. Qual a média de preço de um prato para dois por país?

In [21]:
df1.loc[:, ['average_cost_for_two', 'country']].groupby('country').mean().reset_index().sort_values('average_cost_for_two', ascending=False)

,country,average_cost_for_two
5,Indonesia,303000.000000
0,Australia,138959.783333
11,Sri Lanka,2579.375000
7,Philippines,1227.825000
4,India,704.400514
10,South Africa,339.228324
8,Qatar,174.000000
13,United Arab Emirates,153.716667
9,Singapure,141.437500
1,Brazil,138.812500


## Cidade

### 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [22]:
df1.loc[:, ['restaurant_id', 'city']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0, 0]

'Abu Dhabi'

### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [23]:
df1.loc[df1['aggregate_rating'] >= 4, ['restaurant_id', 'city']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0, 0]

'London'

### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [24]:
df1.loc[df1['aggregate_rating'] <= 2.5, ['restaurant_id', 'city']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0, 0]

'Gangtok'

### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [25]:
df1.loc[:, ['average_cost_for_two', 'city']].groupby('city').mean().reset_index().sort_values('average_cost_for_two', ascending=False).iloc[0, 0]

'Adelaide'

### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?


In [26]:
df1.loc[:, ['cuisines', 'city']].groupby('city').nunique().reset_index().sort_values('cuisines', ascending=False).iloc[0, 0]

'Birmingham'

### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?


In [27]:
df1.loc[df1['has_table_booking'] == 1, ['restaurant_id', 'city']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0, 0]

'Bangalore'

### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [28]:
df1.loc[df1['is_delivering_now'] == 1, ['restaurant_id', 'city']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0, 0]

'Vadodara'

### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?


In [29]:
df1.loc[df1['has_online_delivery'] == 1, ['restaurant_id', 'city']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0, 0]

'Bhopal'

## Restaurantes

### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [30]:
df1.loc[:, ['restaurant_id', 'restaurant_name', 'votes']].sort_values(['votes', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Bawarchi'

### 2. Qual o nome do restaurante com a maior nota média?

In [31]:
df1.loc[:, ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Indian Grill Room'

### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?


In [32]:
df1.loc[:, ['restaurant_id', 'restaurant_name', 'average_cost_for_two']].sort_values(['average_cost_for_two', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

"d'Arry's Verandah Restaurant"

### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [33]:
df1.loc[df1['cuisines'] == 'Brazilian', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Loca Como tu Madre'

### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?


In [34]:
df1.loc[(df1['cuisines'] == 'Brazilian') & (df1['country'] == 'Brazil'), ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Braseiro da Gávea'

### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?


In [35]:
df1.loc[:, ['votes', 'has_online_delivery']].groupby('has_online_delivery').mean().reset_index().sort_values('votes', ascending=False)

,has_online_delivery,votes
1,1,838.821664
0,0,479.421018


Resposta: Sim

### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?


In [36]:
df1.loc[:, ['average_cost_for_two', 'has_table_booking']].groupby('has_table_booking').mean().reset_index().sort_values('average_cost_for_two', ascending=False)

,has_table_booking,average_cost_for_two
1,1,69998.423810
0,0,3488.596866


Resposta: Sim

### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [37]:
df1.loc[(df1['country'] == 'United States of America') & ((df1['cuisines'] == 'Japanese') | (df1['cuisines'] == 'BBQ')), ['average_cost_for_two', 'country', 'cuisines']].groupby(['country', 'cuisines']).mean().reset_index().sort_values('average_cost_for_two', ascending=False)

,country,cuisines,average_cost_for_two
1,United States of America,Japanese,56.406250
0,United States of America,BBQ,39.642857


Resposta: Sim

## Tipos de Culinária

### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [38]:
df1.loc[df1['cuisines'] == 'Italian', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Darshan'

### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [39]:
df1.loc[df1['cuisines'] == 'Italian', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Avenida Paulista'

### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?


In [40]:
df1.loc[df1['cuisines'] == 'American', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Burger & Lobster'

### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [41]:
df1.loc[df1['cuisines'] == 'American', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Alston Bar & Beef'

### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?


In [42]:
df1.loc[df1['cuisines'] == 'Arabian', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Mandi@36'

### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?


In [43]:
df1.loc[df1['cuisines'] == 'Arabian', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Raful'

### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [44]:
df1.loc[df1['cuisines'] == 'Japanese', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Sushi Samba'

### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [45]:
df1.loc[df1['cuisines'] == 'Japanese', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Banzai Sushi'

### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?


In [46]:
df1.loc[df1['cuisines'] == 'Home-made', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Kanaat Lokantası'

### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?


In [47]:
df1.loc[df1['cuisines'] == 'Home-made', ['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'GurMekan Restaurant'

### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [48]:
df1.loc[:, ['average_cost_for_two', 'cuisines']].groupby('cuisines').mean().reset_index().sort_values('average_cost_for_two', ascending=False).iloc[0, 0]

'Modern Australian'

### 12. Qual o tipo de culinária que possui a maior nota média?

In [49]:
df1.loc[:, ['aggregate_rating', 'cuisines']].groupby('cuisines').mean().reset_index().sort_values('aggregate_rating', ascending=False).iloc[0, 0]

'Others'

### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?


In [50]:
df1.loc[(df1['has_online_delivery'] == 1) & (df1['is_delivering_now'] == 1), ['restaurant_id', 'cuisines']].groupby('cuisines').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0, 0]

'North Indian'